In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate

%matplotlib inline

# GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


Using TensorFlow backend.


In [2]:
dataset_dir = "/scratch/wenyuan/Mask-RCNN/Data_Pre_Processing/cedars-224"
held_out_set = 4
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
class InferenceConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)

inference_config = InferenceConfig()
inference_config.display()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights

# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]
# model_path = model.find_specific(epoch = 9, held_out_set = 4)[1]
# model_path = model.find_specific(epoch = 160, held_out_set = held_out_set)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


Configurations:
BACKBONE_SHAPES                [[64 64]
 [32 32]
 [16 16]
 [ 8  8]
 [ 4  4]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        1
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  256
IMAGE_MIN_DIM                  256
IMAGE_PADDING                  True
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.97197787  120.93338008  183.82220684]
MINI_MASK_SHAPE                (56, 56)
NAME                           prostate
NUM_CLASSES                    4
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_

In [9]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)
val_list = [0, 1, 2, 3]
dataset_val.load_prostate(dataset_dir, val_list, mode = 144)
dataset_val.prepare()

In [10]:
def instance_2_sementic(instance_mask, class_ids):
    try:
        h, w, d = instance_mask.shape
    except ValueError:
        mask = int(class_ids) * instance_mask 
        result_dict = {'ATmask': mask}
        return result_dict
    
    mask_map = {}
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = np.logical_or(mask_map[str(label)], \
                                             instance_mask[:, :, index]) \
        if str(label) in mask_map.keys() else instance_mask[:, :, index]
    
    mask = np.zeros((h, w), dtype=np.int)

    for key in mask_map.keys():
        if (key != '0'):
            mask = mask + int(key) * mask_map[key] 
    result_dict = {'ATmask': mask}
    return result_dict

In [11]:
def save_sementic(result_dict, image_id, dataset_dir):
    save_name = \
    os.path.join(dataset_dir, 'masks_sementic_mod/' + str(image_id).zfill(4) + '_sementic.mat')
    scipy.io.savemat(save_name, result_dict)

In [12]:
def prediction_2_sementic(mask, class_ids, scores):
    """ convert maskrcnn prediction to 1 single sementic mask
    """
    mask_map = {} # create a probability dict for each class
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = \
        np.maximum(mask_map[str(label)], scores[index] * mask[:, :, index])\
        if str(label) in mask_map.keys() else scores[index] * mask[:, :, index]
    
    ## convert to h * w * num_classes probability map
    h, w, d = mask.shape
    for i in range(4):
        try:
            sementic_mask = \
            np.concatenate((sementic_mask, \
                            np.expand_dims(mask_map[str(i)], axis = -1)),axis = -1)\
            if (i != 0) else np.expand_dims(mask_map[str(i)], axis = -1)
        except KeyError:
            sementic_mask = np.concatenate((sementic_mask, np.zeros((h, w, 1))),axis = -1)\
            if (i != 0) else np.zeros((h, w, 1))
            
    sementic_mask_res = np.argmax(sementic_mask, axis = -1)
    return sementic_mask_res

In [13]:
def compute_mIOU(c_matrix):
    """ compute the mIOU based on the confusion matrix
    """
    num_class, _ = c_matrix.shape
    IOU = []
    for i in range(num_class):
        p = c_matrix[i, i] / (sum(c_matrix[i, :]) + sum(c_matrix[:, i]) - c_matrix[i, i]) \
        if (sum(c_matrix[i, :]) + sum(c_matrix[:, i]) - c_matrix[i, i]) else 0
        IOU.append(p)
    return np.mean(IOU), IOU

In [15]:
# Compute mIOU as for the 4-classes prostate dataset
# Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 1)
# image_ids = [4]
image_ids = dataset_val.image_ids
C_MATRIX = np.zeros((4, 4))
# exclude_list = [1, 3, 6, 29, 35, 36, 37, 54, 58, 60,
#                61, 63, 68, 76, 79, 80, 84, 85, 88]
exclude_list = []
for image_id in image_ids:
    if image_id in exclude_list:
        continue
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    # Convert gt-instance mask to gt-sementic mask
    gt_sementic_mask = instance_2_sementic(gt_mask, gt_class_id)
    # Whether write the gt-sementic mask
#     save_sementic(gt_sementic_mask, val_list[image_id], dataset_dir)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Convert prediction to sementic mask
    result_sementic = prediction_2_sementic(r['masks'], r['class_ids'], r['scores'])\
    if r['class_ids'].size != 0 else np.zeros((image.shape[0], image.shape[1]))
    # Compute confusion matrix
    c_matrix = confusion_matrix(np.ravel(gt_sementic_mask['ATmask'][412:612, 412:612]), \
                                np.ravel(result_sementic[412:612, 412:612]))
    if c_matrix.size != 16:
        """if the confusion matrix is not 4 by 4
        """
        unique_set = \
        np.union1d(np.unique(result_sementic[412:612, 412:612]), \
                   np.unique(gt_sementic_mask['ATmask'][412:612, 412:612]))
        xv, yv = np.meshgrid(unique_set, unique_set)
        temp = np.zeros((4, 4))
        temp[yv.astype(int), xv.astype(int)] = c_matrix
        c_matrix = temp 
    # Update cofusion matrix
    C_MATRIX = C_MATRIX + c_matrix
    if ((image_id + 1) % 20 == 0):
        print('Done evaluating %d / %d!'%(image_id + 1, len(image_ids)))

# Compute mIOU based on confusion matrix
mIOU, IOU = compute_mIOU(C_MATRIX)
print("mIOU: ", mIOU)

Done evaluating 20 / 576!
Done evaluating 40 / 576!
Done evaluating 60 / 576!
Done evaluating 80 / 576!
Done evaluating 100 / 576!
Done evaluating 120 / 576!
Done evaluating 140 / 576!
Done evaluating 160 / 576!
Done evaluating 180 / 576!
Done evaluating 200 / 576!
Done evaluating 220 / 576!
Done evaluating 240 / 576!
Done evaluating 260 / 576!
Done evaluating 280 / 576!
Done evaluating 300 / 576!
Done evaluating 320 / 576!
Done evaluating 340 / 576!
Done evaluating 360 / 576!
Done evaluating 380 / 576!
Done evaluating 400 / 576!
Done evaluating 420 / 576!
Done evaluating 440 / 576!
Done evaluating 460 / 576!
Done evaluating 480 / 576!
Done evaluating 500 / 576!
Done evaluating 520 / 576!
Done evaluating 540 / 576!
Done evaluating 560 / 576!
mIOU:  0.0


In [17]:
print(image.shape)

(256, 256, 3)


In [ ]:
# Compute mIOU based on confusion matrix
mIOU, IOU = compute_mIOU(C_MATRIX)
print(IOU)

In [ ]:
# scipy.io.savemat('./logs/confusion_matrix_held_out_%d.mat'%held_out_set, {'c_matrix' : C_MATRIX})

In [ ]:
## VISUALIZATION

# def get_ax(rows=1, cols=1, size=8):
#     """Return a Matplotlib Axes array to be used in
#     all visualizations in the notebook. Provide a
#     central point to control graph sizes.
    
#     Change the default size attribute to control the size
#     of rendered images
#     """
#     _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
#     return ax

In [ ]:
# image_id = np.random.choice(dataset_val.image_ids, 1)[0]
# image_id = 4
# original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(dataset_val, inference_config,
#                                image_id, use_mini_mask=False)
# visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
#                             dataset_val.class_names, figsize=(8, 8))
# # Run object detection
# results = model.detect([original_image], verbose=0)
# r = results[0]
# visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset_val.class_names, r['scores'], ax=get_ax())